using https://data.cityofnewyork.us/City-Government/Neighborhood-Names-GIS/99bc-9p23#SODA%20API for neighborhoods

https://github.com/minimaxir/automl-gs machine learning w/ xgboost

In [2]:
# import needed modules

import pandas as pd
import json, requests
import numpy as np
import requests
from geopy.geocoders import Nominatim

from IPython.display import Image
from IPython.core.display import HTML

from pandas.io.json import json_normalize

from sklearn.cluster import KMeans
from kmodes.kmodes import KModes

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium

In [24]:
with open('hoods.geojson') as json_data:
    nyc = json.load(json_data)

In [25]:
hood_data = nyc['features']

In [4]:
hood_data[0]

{'type': 'Feature',
 'properties': {'stacked': '1',
  'name': 'Wakefield',
  'annoline1': 'Wakefield',
  'annoline3': '',
  'objectid': '1',
  'annoangle': '0.0',
  'annoline2': '',
  'borough': 'Bronx'},
 'geometry': {'type': 'Point',
  'coordinates': [-73.8472005205491, 40.89470517661004]}}

In [26]:
columns = ['borough', 'neighborhood', 'latitude', 'longitude']

hoods = pd.DataFrame(columns=columns)

In [27]:
for data in hood_data:
    borough = data['properties']['borough']
    hood_name = data['properties']['name']
    
    hood_latlon = data['geometry']['coordinates']
    hood_lat = hood_latlon[1]
    hood_lon = hood_latlon[0]
    
    hoods = hoods.append({'borough': borough,
                          'Neighborhood': hood_name,
                          'latitude': hood_lat,
                          'longitude': hood_lon}, ignore_index=True)

In [28]:
address = 'New York City, NY'

geoloc = Nominatim(user_agent='ny')
location = geoloc.geocode(address)
lat = location.latitude
lon = location.longitude

In [29]:
map_nyc = folium.Map(location=[lat, lon], zoom_start=15)

for lat, lng, borough, neighborhood in zip(hoods['latitude'], hoods['longitude'], hoods['borough'], hoods['neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nyc)  
    
map_nyc

In [10]:
CLIENT_ID = 'RFQKGJMFRIY0H5WWUHHEN5YHNJMSEPQEQTCQOPMNIQT1R0FF'
CLIENT_SECRET = '3GAEJPFACNZJM2NIG0JZINVV1PIE2O0OTVBVLRX0KC4OVAUQ'
VERSION = '20180605'

In [91]:
def getBurgers(labels, lats, lons):
    
    burgers = []
    
    for label, lat, lon in zip(labels, lats, lons):
        
        print(label)
        
        url =  'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&query=burgers&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lon, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        burgers.append([(
            label,
            lat,
            lon,
            v['venue']['id'],
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    burgers_df = pd.DataFrame([i for burgers in burgers for i in burgers])
    burgers_df.columns = ['Neighborhood', 
              'Neighborhood Latitude', 
              'Neighborhood Longitude',
              'Venue ID',
              'Venue', 
              'Venue Latitude', 
              'Venue Longitude', 
              'Venue Category']

    return(burgers_df)    
    

In [92]:
ny_burgers = getBurgers(labels=hoods['neighborhood'],
                       lats=hoods['latitude'],
                       lons=hoods['longitude'])

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [58]:
ny_burgers.shape

(1218, 7)

In [62]:
# categories for burgers

len(ny_burgers['Venue Category'].unique())

68

In [93]:
ny_burgers.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue ID,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Co-op City,40.874294,-73.829939,4be2b79d660ec9284d04ca3b,Townhouse Restaurant,40.876086,-73.828868,Restaurant
1,Eastchester,40.887556,-73.827806,4c406f2e6c1a9521a042b96a,McDonald's,40.885636,-73.826840,Burger Joint
2,Kingsbridge,40.881687,-73.902818,55568a71498e3524400a356e,Smashburger,40.884221,-73.900333,Burger Joint
3,Kingsbridge,40.881687,-73.902818,4b3789f3f964a520f34125e3,Riverdale Diner,40.885183,-73.899484,Diner
4,Kingsbridge,40.881687,-73.902818,4b9c9c6af964a520b27236e3,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant


In [76]:
most_burgers = ny_burgers.groupby('Neighborhood').count().reset_index().sort_values('Venue', ascending=False)

most_burgers.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
97,Midtown South,61,61,61,61,61,61
96,Midtown,59,59,59,59,59,59
53,Financial District,51,51,51,51,51,51
29,Civic Center,30,30,30,30,30,30
161,Upper West Side,30,30,30,30,30,30


In [16]:
hoods = hoods.rename(columns={'neighborhood': 'Neighborhood'})

hoods

,borough,Neighborhood,latitude,longitude,Neighborhood
0,Bronx,NaN,40.894705,-73.847201,Wakefield
1,Bronx,NaN,40.874294,-73.829939,Co-op City
2,Bronx,NaN,40.887556,-73.827806,Eastchester
3,Bronx,NaN,40.895437,-73.905643,Fieldston
4,Bronx,NaN,40.890834,-73.912585,Riverdale
5,Bronx,NaN,40.881687,-73.902818,Kingsbridge
6,Manhattan,NaN,40.876551,-73.910660,Marble Hill
7,Bronx,NaN,40.898273,-73.867315,Woodlawn
8,Bronx,NaN,40.877224,-73.879391,Norwood
9,Bronx,NaN,40.881039,-73.857446,Williamsbridge


In [6]:
# load csv

ny_burgers = pd.read_csv('merged_data.csv')

ny_burgers.drop(['Unnamed: 0', 'Venue ID'], axis=1, inplace=True)

ny_burgers.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Tip Count,Rating,Like Count
0,Co-op City,40.874294,-73.829939,Townhouse Restaurant,40.876086,-73.828868,Restaurant,5,5.8,3
1,Eastchester,40.887556,-73.827806,McDonald's,40.885636,-73.826840,Burger Joint,10,6.0,1
2,Kingsbridge,40.881687,-73.902818,Smashburger,40.884221,-73.900333,Burger Joint,8,8.4,3
3,Kingsbridge,40.881687,-73.902818,Riverdale Diner,40.885183,-73.899484,Diner,52,7.8,2
4,Kingsbridge,40.881687,-73.902818,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant,19,7.7,3


In [22]:
hoods.drop(['neighborhood'], axis=1, inplace=True)

In [7]:
machine_learning_data = pd.get_dummies(ny_burgers[['Venue Category', 'Neighborhood']], prefix="", prefix_sep="")

machine_learning_data[['Tip Count', 'Rating', 'Like Count']] = ny_burgers[['Tip Count', 'Rating', 'Like Count']]



In [47]:
machine_learning_data.head()

machine_learning_data.to_csv('auto_ml.csv')

In [8]:
machine_learning_data.shape

(382, 125)

In [9]:
pd.read_csv('automl_results.csv')

,trial_id,epoch,time_completed,mse,mae,r_2,base_lr,numeric_strat,categorical_strat,max_depth,gamma,min_child_weight,subsample,colsample_bytree,max_bin,reg_objective
0,e9709a0b-4b62-454b-afaa-d23c1a32469b,1,2019-05-10 01:44:53,48.353591,6.891370,-54.819910,0.0100,minmax,all_binary,9,5,1,1.0,1.0,512,reg:linear
1,e9709a0b-4b62-454b-afaa-d23c1a32469b,2,2019-05-10 01:44:53,47.429882,6.824281,-53.753571,0.0100,minmax,all_binary,9,5,1,1.0,1.0,512,reg:linear
2,e9709a0b-4b62-454b-afaa-d23c1a32469b,3,2019-05-10 01:44:53,46.508328,6.756509,-52.689719,0.0100,minmax,all_binary,9,5,1,1.0,1.0,512,reg:linear
3,e9709a0b-4b62-454b-afaa-d23c1a32469b,4,2019-05-10 01:44:53,45.620615,6.690753,-51.664933,0.0100,minmax,all_binary,9,5,1,1.0,1.0,512,reg:linear
4,e9709a0b-4b62-454b-afaa-d23c1a32469b,5,2019-05-10 01:44:53,44.750272,6.625649,-50.660199,0.0100,minmax,all_binary,9,5,1,1.0,1.0,512,reg:linear
5,e9709a0b-4b62-454b-afaa-d23c1a32469b,6,2019-05-10 01:44:53,43.881817,6.559869,-49.657646,0.0100,minmax,all_binary,9,5,1,1.0,1.0,512,reg:linear
6,e9709a0b-4b62-454b-afaa-d23c1a32469b,7,2019-05-10 01:44:53,43.045382,6.496057,-48.692057,0.0100,minmax,all_binary,9,5,1,1.0,1.0,512,reg:linear
7,e9709a0b-4b62-454b-afaa-d23c1a32469b,8,2019-05-10 01:44:53,42.225311,6.432879,-47.745357,0.0100,minmax,all_binary,9,5,1,1.0,1.0,512,reg:linear
8,e9709a0b-4b62-454b-afaa-d23c1a32469b,9,2019-05-10 01:44:53,41.406883,6.369032,-46.800555,0.0100,minmax,all_binary,9,5,1,1.0,1.0,512,reg:linear
9,e9709a0b-4b62-454b-afaa-d23c1a32469b,10,2019-05-10 01:44:53,40.618758,6.307108,-45.890735,0.0100,minmax,all_binary,9,5,1,1.0,1.0,512,reg:linear


In [11]:
from automl_gs import automl_grid_search

automl_grid_search('auto_ml.csv', 'Rating')

Solving a regression problem, minimizing mse using tensorflow.

Modeling with field specifications:
Unnamed: 0: numeric
American Restaurant: categorical
Asian Restaurant: categorical
Australian Restaurant: categorical
BBQ Joint: categorical
Bakery: categorical
Bar: categorical
Beer Garden: categorical
Breakfast Spot: categorical
Brewery: categorical
Burger Joint: categorical
Café: categorical
Cocktail Bar: categorical
Coffee Shop: categorical
Deli / Bodega: categorical
Diner: categorical
Dive Bar: categorical
Eastern European Restaurant: categorical
Fast Food Restaurant: categorical
Food & Drink Shop: categorical
Food Truck: categorical
French Restaurant: categorical
Fried Chicken Joint: categorical
Gastropub: categorical
Greek Restaurant: categorical
Hookah Bar: categorical
Hot Dog Joint: categorical
Hotel Bar: categorical
Irish Pub: categorical
Italian Restaurant: categorical
Juice Bar: categorical
Lounge: categorical
Mediterranean Restaurant: categorical
New American Restaurant: cat

FileNotFoundError: [Errno 2] File b'automl_train/metadata/results.csv' does not exist: b'automl_train/metadata/results.csv'

In [12]:
automl_grid_search('auto_ml.csv', 'Rating', framework='xgboost', num_epochs=10)



Solving a regression problem, minimizing mse using xgboost.

Modeling with field specifications:
Unnamed: 0: numeric
American Restaurant: categorical
Asian Restaurant: categorical
Australian Restaurant: categorical
BBQ Joint: categorical
Bakery: categorical
Bar: categorical
Beer Garden: categorical
Breakfast Spot: categorical
Brewery: categorical
Burger Joint: categorical
Café: categorical
Cocktail Bar: categorical
Coffee Shop: categorical
Deli / Bodega: categorical
Diner: categorical
Dive Bar: categorical
Eastern European Restaurant: categorical
Fast Food Restaurant: categorical
Food & Drink Shop: categorical
Food Truck: categorical
French Restaurant: categorical
Fried Chicken Joint: categorical
Gastropub: categorical
Greek Restaurant: categorical
Hookah Bar: categorical
Hot Dog Joint: categorical
Hotel Bar: categorical
Irish Pub: categorical
Italian Restaurant: categorical
Juice Bar: categorical
Lounge: categorical
Mediterranean Restaurant: categorical
New American Restaurant: catego


Metrics:
trial_id: 8d7618bb-76da-4397-a976-08ef34423549
epoch: 10
time_completed: 2019-05-10 03:09:59
mse: 40.661996790570676
mae: 6.311002240388288
r_2: -45.9406504116858


Metrics:
trial_id: 1a813c90-6482-4c52-9d76-ef30d218f1f0
epoch: 10
time_completed: 2019-05-10 03:10:12
mse: 40.60989684069932
mae: 6.3042568252397615
r_2: -45.880505664098166


Metrics:
trial_id: c023362e-d8dd-440a-a9a3-929cec90fd91
epoch: 10
time_completed: 2019-05-10 03:10:38
mse: 6.974868305119633
mae: 2.495041480686354
r_2: -7.051863671734189


Metrics:
trial_id: d0cecee6-6918-4e9c-b6ef-212efbfe45ab
epoch: 10
time_completed: 2019-05-10 03:11:26
mse: 6.927548716896868
mae: 2.474805104628853
r_2: -6.997237425516371


Metrics:
trial_id: a6675323-e119-419e-8c78-0babafeb9826
epoch: 10
time_completed: 2019-05-10 03:14:56
mse: 6.815604259589907
mae: 2.468751422633296
r_2: -6.868007529034985


In [13]:
df_train = pd.read_csv('auto_ml.csv')

In [14]:
from auto_ml import Predictor

column_descriptions = {
  'Rating': 'output'
}

ml_predictor = Predictor(type_of_estimator='regressor', column_descriptions=column_descriptions)

ml_predictor.train(df_train)

/anaconda3/lib/python3.7/site-packages/deap/tools/_hypervolume/pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)


Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at http://auto.ml
You are running on version 2.9.10
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}
Running basic data cleaning
Fitting DataFrameVectorizer
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'presort': False, 'learning_rate': 0.1, 'warm_start': True}


********************************************************************************************
About to fit the pipeline for the model GradientBoostingRegressor to predict Rating
Started at:
20